In [15]:
import copy
import numpy as np
import mlx
import mlx.nn as mx
import mlx.core as mx_core
import torch
import torch.nn as nn
import torch.nn.functional as F
import safetensors
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from huggingface_hub import notebook_login
from pprint import pprint

# 테스트 모델 Gemma-2b-it 업로드
path = "./model/Gemma-2b-it/"
model_name = "model-{}-of-{}.safetensors"
model1 = safetensors.safe_open(path+model_name.format("00001", "00002"), framework="pt")
model2 = safetensors.safe_open(path+model_name.format("00002", "00002"), framework="pt")

print(model1.keys())
print(model2.keys())

# model1, model2의 tensor들을 답음
tensors = {}
for key in model1.keys():
    tensors[key] = model1.get_tensor(key)
for key in model2.keys():
    tensors[key] = model2.get_tensor(key)
for key in tensors:
    print(key, tensors[key].shape)

['model.embed_tokens.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.10.input_layernorm.weight', 'model.layers.10.mlp.down_proj.weight', 'model.layers.10.mlp.gate_proj.weight', 'model.layers.10.mlp.up_proj.weight', 'model.layers.10.post_attention_layernorm.weight', 'model.laye

In [16]:
# torch와 mlx 간의 weight가 서로 다름
inputs = torch.randn(200, 100)
layer1 = nn.Linear(100, 200, dtype=torch.float32).requires_grad_(False)
layer2 = mx.Linear(100, 200)
output1 = layer1(inputs)
layer2.weight = mx_core.array(layer1.weight.numpy())
layer2.bias   = mx_core.array(layer1.bias.numpy())
output2 = layer2(mlx.core.array(inputs))
print(output1)
print(output2)

tensor([[ 1.0233,  0.9686, -0.3742,  ..., -0.6960, -0.3375, -0.1172],
        [-0.7276, -0.7022, -1.0547,  ...,  0.3612, -0.3441,  0.3949],
        [-0.8054,  0.3337,  0.1827,  ..., -0.0143,  0.4280,  0.4908],
        ...,
        [ 0.2764, -0.6819, -1.0033,  ..., -0.2844, -0.4962, -0.8181],
        [ 0.7598, -0.0205, -0.2107,  ...,  0.1204, -1.0318,  0.1993],
        [-0.1827, -0.6451, -0.1667,  ..., -0.1079, -0.4077, -1.1762]])
array([[1.02331, 0.968554, -0.374168, ..., -0.695993, -0.33745, -0.117239],
       [-0.727575, -0.702247, -1.0547, ..., 0.361208, -0.344141, 0.394925],
       [-0.805406, 0.333706, 0.182659, ..., -0.0142567, 0.427993, 0.490802],
       ...,
       [0.276407, -0.681926, -1.00328, ..., -0.284429, -0.496206, -0.818095],
       [0.759834, -0.0204772, -0.210725, ..., 0.120362, -1.03176, 0.199336],
       [-0.182736, -0.645108, -0.166685, ..., -0.107906, -0.407675, -1.17617]], dtype=float32)


In [32]:
hidden_size=2048
intermediate_size=16384
inputs = torch.randn(hidden_size, intermediate_size)
weight1 = nn.Parameter(torch.empty(hidden_size, intermediate_size))
weight2 = nn.Parameter(torch.empty(hidden_size, intermediate_size))
weight3 = nn.Parameter(torch.empty(intermediate_size, hidden_size))

gate = F.linear(inputs, weight1)
print(gate.shape)
up   = F.linear(inputs, weight2)
print(up.shape)
fuse = gate * up
print(fuse.shape, weight3.shape)
down = F.linear(fuse, weight3)

torch.Size([2048, 2048])
torch.Size([2048, 2048])
torch.Size([2048, 2048]) torch.Size([16384, 2048])
